# Training (Optional)

This section explains how to train a CNN on OASIS data that was processed in the previous sections. However if you do not have an access to a GPU, training the CNN will require too much time, this is why pretrained models has be given to execute the previous part of the course (inference).

In [ ]:
import torch

# Check if a GPU is available
print('GPU is available', torch.cuda.is_available())

# You can execute this notebook on Colab to run it on a GPU.

In this notebook the goal is to train a classifier to differentiate the **AD label from the CN label** based on the associated **T1-MR images**. These images were preprocessed according to the <font color='green'>appropriate section</font>.

According to the literature review done in <font color='green'>give here the reference of AD-DL</font>, 4 main paradigms were implemented to train a CNN to perform a classification task between AD and CN labels based on T1-MRI. These paradigms depend of the input given to the network:
* 2D slices,
* 3D patches,
* ROIs (Regions of Interest).
* 3D images.

There is no consensus in the literature on the architecture of the models that should be used for each input category. Part of the studies of the bibliography used custom models, and the other reused architectures that led to good results on natural images (VGGNet, ResNet, GoogleNet, DenseNet...). We chose to find custom architectures by running a search on the main architecture components (more details in <font color='green'>give here the reference of relevant section of AD-DL</font>). This architecture search was run only on the train + validation sets, and **no test sets were used to choose the best architecture**. This is crucial, as the use of a test set during architecture search can lead to over-optimistic results.

<div class="alert alert-block alert-info">
<b>Computational issues:</b><p>
    Depending on your computational ressources, you should adapt the parameters in <b>COMPUTATIONAL ISSUES</b> group:
    <ul>
        <li> -cpu flag forces the system to use CPU instead of looking for a GPU. </li>
        <li> --nproc N sets the number of workers (parallel processes) in the Dataloader to N. </li>
        <li> --batch_size B will set the batch size to B. If the batch size is too high, it can raise memory errors. </li>
    </ul>
</div>

## 2D slice-level CNN

An advantage of this approach is that existing CNNs which had huge success for natural image classification, e.g. ResNet [(He et al., 2016)](https://doi.org/10.1109/CVPR.2016.90) and VGGNet [(Simonyan and Zisserman, 2014)](https://arxiv.org/abs/1409.1556) , can be easily borrowed and used in a transfer learning fashion. Other advantages are the increased number of training samples as many slices can be extracted from a single 3D image, and a lower memory usage compared to using the full MR image as input.

The 2D slice-level CNN in `clinicadl` is a slight modification of the ResNet-18 network used to train natural images on the ImageNet dataset:
* One fully-connected layer was added at the end of the network to reduce the dimension of the output from 1000 to 2 classes <font color='purple'>(purple dotted box)</font>.
* The last five convolutional layers and the last FC of ResNet are fine-tuned <font color='green'>(green dotted box)</font>.
* All other layers have their weights and biases fixed.

<img src="./images/2DCNN.png">

During training, the gradients update are done based on the loss computed on the slice level. Final performances are computed on the subject level by combining the outputs of the slices of the same subject.

<div class="alert alert-block alert-success">
<b>Transfer learning:</b> 
    <p>The layers of this network (except the last one) are initialized with the weights of the ResNet-18 trained on ImageNet.</p>
</div>

In [ ]:
!clinicadl train slice -h
#!clinicadl train slice <caps_dir> data/labels_list results/slice_cnn resnet18 --batch_size 16 --use_extracted_slices

## 3D-patch level & ROI-based models

The 3D patch-level models compensate the absence of 3D information in the 2D slice-level approach and keep some of its advantages (low memory usage and larger sample size). This paradigm can be divided in two different frameworks:
- `single-CNN`: one CNN is trained on all patch locations.
- `multi-CNN`: one CNN is trained per patch location.
For `multi-CNN` the sample size is smaller (equivalent to image level framework), however the CNNs may be more accurate as they are specialized in one patch location.

ROI-based models are similar to 3D-patch but take advantage of prior knowledge on Alzheimer's disease. Indeed most of the patches are not informative as they contain parts of the brain that are not affected by the disease. Methods based on regions of interest (ROI) overcome this issue by focusing on regions which are known to be informative: the hippocampi. In this way, the complexity of the framework can be decreased as fewer inputs are used to train the networks.

The 3D patch-level and ROI-based CNNs in `clinicadl` have the same architecture, including:
* 4 convolutional layers with kernel 3x3x3,
* 4 max pooling layers with stride and kernel of 2 and a padding value that automatically adapts to the input feature map size.
* 3 fully-connected layers.

For 3D-patch level networks, it is possible to train one CNN per patch location. In this case use the subparser `multicnn` and specify the number of patches in `--num_cnn`.

<img src="./images/ROICNN.png">

As for the 2D slice-level model, the gradients update are done based on the loss computed on the slice level. Final performances are computed on the subject level by combining the outputs of the patches or the two hippocampi of the same subject.

<div class="alert alert-block alert-success">
<b>Transfer learning:</b> 
    <p>It is possible for this category to train an autoencoder derived from the CNN architecture. The encoder will share the same architecture than the CNN until the fully-connected layers.</p>
    <img src="./images/autoencoder.png">
    <p>Then the weights of the encoder will be transferred in the convolutions of the CNN to initialize it before its training. This procedure is called <i>autoencoder pretraining</i>.</p>
    <p>It is also possible to transfer weights between two CNNs with the same architecture.</p>
    <p>For 3D-patch multi-CNNs specifically, it is possible to initialize each CNN of a multi-CNN:
        <ul>
        <li> with the weights of a single-CNN,</li>
        <li> with the weights of the corresponding CNN in a multi-CNN. </li>
    </ul>
    <p>Transferring weights between CNNs can be useful when performing two classification tasks that are similar. This is what has been done in <font color='green'>give here the reference of AD-DL</font>: the sMCI vs pMCI classification network was initialized with the weights of the AD vs CN classification network.</p>
</div>

<div class="alert alert-block alert-danger">
<b>Transfer learning:</b> 
    <p>Transferring weights between tasks that are not similar enough can hurt the performance !</p>
</div>

### 3D-patch level models

See definition of patches in the <font color="green">extract</font> section of the course.

In [ ]:
# 3D-patch autoencoder pretraining
!clinicadl train patch autoencoder -h
#!clinicadl train patch autoencoder <caps_dir> data/labels_list results/patch_autoencoder Conv4_FC3 --batch_size 16 --use_extracted_patches

In [ ]:
# 3D-patch single-CNN training
!clinicadl train patch cnn -h

# With autoencoder pretraining
#!clinicadl train patch cnn <caps_dir> data/labels_list results/patch_single-cnn Conv4_FC3 --batch_size 16 --use_extracted_patches --transfer_learning_autoencoder --transfer_learning_path results/patch_autoencoder
# Without pretraining
#!clinicadl train patch cnn <caps_dir> data/labels_list results/patch_single-cnn Conv4_FC3 --batch_size 16 --use_extracted_patches

In [ ]:
# 3D-patch multi-CNN training
!clinicadl train patch multicnn -h

# With autoencoder pretraining
#!clinicadl train patch multicnn <caps_dir> data/labels_list results/patch_multi-cnn Conv4_FC3 --batch_size 16 --use_extracted_patches --transfer_learning_autoencoder --transfer_learning_path results/patch_autoencoder
# With single-CNN pretraining
#!clinicadl train patch multicnn <caps_dir> data/labels_list results/patch_multi-cnn Conv4_FC3 --batch_size 16 --use_extracted_patches --transfer_learning_path results/patch_single-cnn
# Without pretraining
#!clinicadl train patch multicnn <caps_dir> data/labels_list results/patch_multi-cnn Conv4_FC3 --batch_size 16 --use_extracted_patches

### ROI-based models

See definition of ROI in the <font color="green">extract</font> section of the course.

In [ ]:
# ROI-based autoencoder pretraining
!clinicadl train roi autoencoder -h
#!clinicadl train roi autoencoder <caps_dir> data/labels_list results/roi_autoencoder Conv4_FC3 --batch_size 16

In [ ]:
# ROI-based CNN training
!clinicadl train roi cnn -h

# With autoencoder pretraining
#!clinicadl train roi cnn <caps_dir> data/labels_list results/roi_cnn Conv4_FC3 --batch_size 16 --transfer_learning_autoencoder --transfer_learning_path results/roi_autoencoder
# Without pretraining
#!clinicadl train roi cnn <caps_dir> data/labels_list results/roi_cnn Conv4_FC3 --batch_size 16

## 3D-image level model

In this approach, the whole MRI is used at once and the classification is performed at the imageg level. The advantage is that the spatial information is fully integrated, and it may allow to discover new knowledge on the disease. However, it requires more computational resources (especially GPU with higher memory capacity).

The 3D image-level CNN in `clinicadl` is designed as follows:
* 5 convolutional layers with kernel 3x3x3,
* 5 max pooling layers with stride and kernel of 2 and a padding value that automatically adapts to the input feature map size.
* 3 fully-connected layers.

<img src="./images/imageCNN.png">

Depending on the preprocessing, the size of the fully connected layers must be adapted. This is why two models exist in `clinicadl`:
* `Conv5_FC3` adapted to `linear` preprocessing, <font color="green">names might change</font>
* `Conv5_FC3_mni` adapted to `mni` preprocessing.


<div class="alert alert-block alert-success">
<b>Transfer learning:</b> 
    <p>It is possible for this category to train an autoencoder derived from the CNN architecture, or to transfer weights between CNNs. See the section on patches for more details on this topic !</p>
</div>

In [ ]:
# 3D-image level autoencoder pretraining
!clinicadl train image autoencoder -h
#!clinicadl train image autoencoder <caps_dir> data/labels_list results/image_autoencoder Conv5_FC3 -gpu

In [ ]:
# 3D-image level autoencoder pretraining
!clinicadl train image cnn -h

# With autoencoder pretraining
#!clinicadl train image cnn <caps_dir> data/labels_list results/image_cnn Conv5_FC3 --transfer_learning_autoencoder --transfer_learning_path results/image_autoencoder
# Without pretraining
#!clinicadl train image cnn <caps_dir> data/labels_list results/image_cnn Conv5_FC3

## Custom models

You want to add your custom architecture and train it with clinicadl ? Please fork and clone the [github repo](https://github.com/aramis-lab/AD-DL]) and code your model in pytorch in `clinicadl/tools/deep_learning/models` and import it in `clinicadl/tools/deep_learning/models/__init__.py` !